Import

In [ ]:
!pip install opencv-python
!pip install pingouin

In [ ]:
#Import Python Packages
import copy;
import cv2;
from functools import reduce
from glob import glob
from glob import iglob
import logging
import math
import matplotlib.pyplot as plt

import numpy as np
from operator import __add__
import os
from os.path import splitext
from os import listdir
import pandas as pd;
from PIL import Image, ImageOps
import PIL
import pingouin as pg
from random import randint
import random;
import sys
from scipy.stats import wilcoxon
from scipy import ndimage
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.measurements import label
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support as score
from skimage import exposure
from skimage import feature
from skimage import transform as tf
from skimage import morphology
from skimage.morphology import skeletonize
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch import optim
from torch.nn import Parameter
from torch.nn.modules import Conv2d, Module
from tqdm import tqdm
from typing import Any
import warnings
warnings.filterwarnings('ignore')

Neural Network

In [ ]:
class GaborConv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=48, dilation=1,groups=1,bias=False, padding_mode="reflect"): 
        super().__init__()
        self.is_calculated = False

        self.conv_layer = Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias, padding_mode)
        self.kernel_size = self.conv_layer.kernel_size

        # small addition to avoid division by zero
        self.delta = 1e-3

        #Frequency
        self.freq = Parameter(
            (math.pi / 2)
            * math.sqrt(2)
            ** (-torch.randint(0, 5, (out_channels, in_channels))).type(torch.Tensor),
            requires_grad=True,
        )
        #Theta
        self.theta = Parameter(
            (math.pi / 8)
            * torch.randint(0, 8, (out_channels, in_channels)).type(torch.Tensor),
            requires_grad=True,
        )
        #Sigma
        self.sigma = Parameter(math.pi / self.freq, requires_grad=True)

        #Psi
        self.psi = Parameter(
            math.pi * torch.rand(out_channels, in_channels), requires_grad=True
        )

        self.x0 = Parameter(
            torch.ceil(torch.Tensor([self.kernel_size[0] / 2]))[0], requires_grad=False
        )
        self.y0 = Parameter(
            torch.ceil(torch.Tensor([self.kernel_size[1] / 2]))[0], requires_grad=False
        )

        self.y, self.x = torch.meshgrid(
            [
                torch.linspace(-self.x0 + 1, self.x0 + 0, self.kernel_size[0]),
                torch.linspace(-self.y0 + 1, self.y0 + 0, self.kernel_size[1]),
            ]
        )
        self.y = Parameter(self.y.clone())
        self.x = Parameter(self.x.clone())

        self.weight = Parameter(
            torch.empty(self.conv_layer.weight.shape, requires_grad=True),
            requires_grad=True,
        )

        self.register_parameter("freq", self.freq)
        self.register_parameter("theta", self.theta)
        self.register_parameter("sigma", self.sigma)
        self.register_parameter("psi", self.psi)
        self.register_parameter("x_shape", self.x0)
        self.register_parameter("y_shape", self.y0)
        self.register_parameter("y_grid", self.y)
        self.register_parameter("x_grid", self.x)
        self.register_parameter("weight", self.weight)

    def forward(self, input_tensor):
        if self.training:
            self.calculate_weights()
            self.is_calculated = False
        if not self.training:
            if not self.is_calculated:
                self.calculate_weights()
                self.is_calculated = True
        return self.conv_layer(input_tensor)

    def calculate_weights(self):
        for i in range(self.conv_layer.out_channels):
            for j in range(self.conv_layer.in_channels):
                sigma = self.sigma[i, j].expand_as(self.y) 
                freq = self.freq[i, j].expand_as(self.y) 
                theta = self.theta[i, j].expand_as(self.y) 
                psi = self.psi[i, j].expand_as(self.y) 

                rotx = self.x * torch.cos(theta) + self.y * torch.sin(theta)
                roty = -self.x * torch.sin(theta) + self.y * torch.cos(theta)

                g = torch.exp(
                    -0.5 * ((rotx ** 2 + roty ** 2) / (sigma + self.delta) ** 2)
                )
                g = g * torch.cos(freq * rotx + psi)
                g = g / (2 * math.pi * sigma ** 2)
                self.conv_layer.weight.data[i, j] = g

    def _forward_unimplemented(self, *inputs: Any):
        """
        code checkers makes implement this method,
        looks like error in PyTorch
        """
        raise NotImplementedError
        
        

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class filt_cat(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

    def forward(self, x1, x2):     

        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return x;

class SNP_Net(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(SNP_Net, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.g0 = GaborConv2d(n_channels, n_channels, kernel_size=(96, 96))
        self.fc = filt_cat(n_channels, 2*n_channels)
        self.inc = DoubleConv(2*n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)



    def forward(self, x):
        f = self.g0(x)
        x0 = self.fc(f,x);
        x1 = self.inc(x0)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        m = nn.Softmax(dim=1)
        logits = m(x)
        return logits


Segmentation Metrics

In [ ]:
def segmentation_metrics(confusion_matrix, per_class=False):
    ret = {
        'mIoU': confusion_matrix.get_mean_class_iou(),
        'mDice': confusion_matrix.get_mean_class_dice(),
        'oAcc': confusion_matrix.get_overall_accuracy(),
        'mSens': confusion_matrix.get_mean_class_sensitivity(),
        'mSpec': confusion_matrix.get_mean_class_specificity(),
        'mF1': confusion_matrix.get_mean_class_f1(),
    }
    if per_class:
        for c,v in enumerate(confusion_matrix.get_dice_per_class()):
            ret['Dice{}'.format(c)] = v
        for c,v in enumerate(confusion_matrix.get_sensitivity_per_class()):
            ret['Sens{}'.format(c)] = v
        for c,v in enumerate(confusion_matrix.get_specificity_per_class()):
            ret['Spec{}'.format(c)] = v
        for c,v in enumerate(confusion_matrix.get_f1_per_class()):
            ret['F1{}'.format(c)] = v
        for c,v in enumerate(confusion_matrix.get_ground_truth_ratios()):
            ret['RGt{}'.format(c)] = v
        for c,v in enumerate(confusion_matrix.get_prediction_ratios()):
            ret['RPr{}'.format(c)] = v
    return ret


class ConfusionMatrix():
    """Maintains a running confusion matrix for a K-class classification problem.
    Rows corresponds to ground-truth targets and columns corresponds to predicted targets."""

    def __init__(self, k):
        self.cm = np.zeros(shape=(k, k), dtype=object)  # array of python ints for unlimited precision
        self.k = k

    def reset(self):
        self.cm.fill(0)

    def add(self, predicted, target):
        """
        Adds new results to the confusion matrix. Filters elements without ground truth (class = -100).

        :param predicted: N, BHW or BDHW-sized tensor of class integers or NK, BKHW or BKDHW-sized tensor
                          of predicted probabilities
        :param target: N, BHW or BDHW-sized tensor of class integers or NK, BKHW or BKDHW-sized tensor
                          of one-hot encoded classes
        """
        # TODO: rewrite into pytorch for speed (no need to copy back to device)

        if isinstance(predicted, torch.Tensor):
            predicted = predicted.detach().cpu().numpy()
        if isinstance(target, torch.Tensor):
            target = target.detach().cpu().numpy()

        if np.ndim(predicted) > 1 and predicted.shape[1] == self.k:
            predicted = np.argmax(predicted, 1)
        else:
            assert (predicted.max() < self.k) and (predicted.min() >= 0), \
                'predicted values are not between 0 and k-1'

        if np.ndim(target) > 1 and target.shape[1] == self.k:
            assert (target >= 0).all() and (target <= 1).all(), \
                'in one-hot encoding, target values should be 0 or 1'
            invalid_idx = target.sum(1) < 0.5
            target = np.argmax(target, 1)
            target[invalid_idx] = -100

        predicted = np.ravel(predicted)
        target = np.ravel(target)
        assert predicted.shape[0] == target.shape[0], \
            'number of targets and predicted outputs do not match'

        # Remove predictions for elements without ground truth
        valid_idx = target != -100
        target = target[valid_idx]
        predicted = predicted[valid_idx]

        # from https://github.com/pytorch/tnt/blob/master/torchnet/meter/confusionmeter.py
        # (sklearn.metrics.confusion_matrix is 100x slower)

        
        x = predicted + self.k * target
 
        
        bincount_2d = np.bincount(x.astype(np.int64),
                                  minlength=self.k ** 2)
        
        
        assert bincount_2d.size == self.k ** 2
        cm = bincount_2d.reshape((self.k, self.k))

        self.cm += cm

    def value(self, normalized=False):
        """
        :return confusion matrix of K rows and K columns
        """
        conf = self.cm.astype(np.float64)
        if normalized:
            return conf / conf.sum(1).clip(min=1e-12)[:, None]
        else:
            return conf

    def get_iou_per_class(self):
        """
        :return per-class intersection-over-union / Jaccard coefficient (NaN if class not present nor predicted)
        """
        cm = self.value()
        tp = np.diag(cm)
        rowsum = cm.sum(axis=0)
        colsum = cm.sum(axis=1)
        with np.errstate(invalid='ignore'):
            return tp / (rowsum + colsum - tp)

    def get_mean_class_iou(self):
        """
        :return mean per-class intersection-over-union / Jaccard coefficient (over classes present)
        """
        iou = self.get_iou_per_class()
        iou = iou[~np.isnan(iou)]
        return np.mean(iou)

    def get_overall_accuracy(self):
        """
        :return overall sensitivity (class-unspecific)
        """
        cm = self.value()
        tp = np.diag(cm).sum()
        oa = tp / max(1, np.sum(cm))
        return oa

    def get_sensitivity_per_class(self):
        """
        :return per-class sensitivity (NaN if class not present)
        """
        cm = self.value()
        tp = np.diag(cm)
        with np.errstate(invalid='ignore'):
            return tp / np.sum(cm, axis=1)

    def get_mean_class_sensitivity(self):
        """
        :return mean per-class sensitivity (over classes present)
        """
        sens = self.get_sensitivity_per_class()
        sens = sens[~np.isnan(sens)]
        return np.mean(sens)


    def get_specificity_per_class(self):
        """
        :return per-class specificity (NaN if class not present)
        """
        cm = copy.deepcopy(self.value())
        tp = np.diag(copy.deepcopy(cm))
        tn = copy.deepcopy(tp);
        for i in range(0,len(tn)):
          tn[i] = tp.sum() - tp[i];
          cm[i,i] = 0;

        with np.errstate(invalid='ignore'):
            return tn /(tn +  np.sum(cm, axis=0))

    def get_mean_class_specificity(self):
        """
        :return mean per-class specificity (over classes present)
        """
        spec = self.get_specificity_per_class()
        spec = spec[~np.isnan(spec)]
        return np.mean(spec)

    def get_f1_per_class(self):
        """
        :return per-class f1 (NaN if class not present)
        """
        cm = self.value()
        tp = np.diag(cm)
        with np.errstate(invalid='ignore'):
            return tp / (0.5*(np.sum(cm, axis=1) + np.sum(cm, axis=0)))


    def get_mean_class_f1(self):
        """
        :return mean per-class f1 (over classes present)
        """
        f1 = self.get_f1_per_class()
        f1 = f1[~np.isnan(f1)]
        return np.mean(f1)



    def get_dice_per_class(self):
        """
        :return per-class Dice coefficient (NaN if class not present nor predicted)
        """
        with np.errstate(invalid='ignore'):
            jacc = self.get_iou_per_class()
            return 2 * jacc / (jacc + 1)

    def get_mean_class_dice(self):
        """
        :return mean per-class Dice coefficient (over classes present)
        """
        dice = self.get_dice_per_class()
        dice = dice[~np.isnan(dice)]
        return np.mean(dice)

Weighted Average and StDev

In [ ]:
def weighted_avg_and_std(values, weights):

    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)
    
    return (average, math.sqrt(variance))

Gaussian Filter

In [ ]:
def fspecial_gauss(size, sigma):

    """Function to mimic the 'fspecial' gaussian MATLAB function
    """

    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g



Find Branchpoints

In [ ]:
def skeleton_branchpoints(skel):
    # Make our input nice, possibly necessary.
    skel = skel.copy()
    skel[skel!=0] = 1
    skel = np.uint8(skel)

    # Apply the convolution.
    kernel = np.uint8([[1,  1, 1],
                       [1, 10, 1],
                       [1,  1, 1]])
    src_depth = -1
    filtered = cv2.filter2D(skel,src_depth,kernel)


    out = np.zeros_like(skel)
    out[np.where(filtered>12)] = 1
    return out

Find Endpoints

In [ ]:
def skeleton_endpoints(skel):
    # Make our input nice, possibly necessary.
    skel = skel.copy()
    skel[skel!=0] = 1
    skel = np.uint8(skel)
    
    skel = np.pad(skel, (1, 1), 'constant', constant_values=(0, 0))

    # Apply the convolution.
    kernel = np.uint8([[1,  1, 1],
                       [1, 10, 1],
                       [1,  1, 1]])
    src_depth = -1
    filtered = cv2.filter2D(skel,src_depth,kernel)

    # Look through to find the value of 11.
    # This returns a mask of the endpoints, but if you
    # just want the coordinates, you could simply
    # return np.where(filtered==11)
    out = np.zeros_like(skel)
    out[np.where(filtered==11)] = 1
    out = out[1:-1,1:-1]

    return out

Count Immune

In [ ]:
def count_immune_cells(mask, mask_AAR):

    #Binary Mask of Immune Cells
    binary = np.zeros(np.shape(mask)) 
    binary[np.where(mask==2)] = 1;


    #Create Unique labels
    islands = np.zeros(np.shape(binary))              
    structure = np.ones((3, 3), dtype=int) 
    labeled, ncomponents = label(binary, structure)
    labeled = labeled*binary;

    #Count Immune Cells
    count = 0;
    unique = np.unique(labeled)
    for unique_label in unique:
        if(unique_label!=0):
            if(np.shape(np.where(labeled ==unique_label))[1]>9):
                count = count + 1;    
    

    #Normalize by Area of SNP
    count = count * np.size(mask_AAR)/np.shape(np.where(mask_AAR==1))[1] 
  
    return count;


Count Neuromas

In [ ]:
def count_neuromas(mask, mask_AAR):

    #Binary Mask of Neuromas
    binary = np.zeros(np.shape(mask)) 
    binary[np.where(mask==3)] = 1;


    #Create Unique labels
    islands = np.zeros(np.shape(binary))              
    structure = np.ones((3, 3), dtype=int) 
    labeled, ncomponents = label(binary, structure)
    labeled = labeled*binary;

    #Count Neuromas
    count = 0;
    unique = np.unique(labeled)
    for unique_label in unique:
        if(unique_label!=0):
            if(np.shape(np.where(labeled ==unique_label))[1]>9):
                count = count + 1;    
    

    #Normalize by Area of SNP
    count = count * np.size(mask_AAR)/np.shape(np.where(mask_AAR==1))[1];
  
    return count;

Count Junctions

In [ ]:
def count_junctions(mask, mask_AAR):
    
  
    #Remove Holes
    skel = skeletonize(mask==1);
    invert_skel = skel==0;
    invert_skel = np.array(morphology.remove_small_objects(invert_skel, 10)) 
    skel = np.abs(invert_skel-1)
    
    #Skeletonize
    skel = skeletonize(skel)

    #Remove Boundarys
    skel[0,:] = 0;
    skel[:,0] = 0;
    skel[-1,:] = 0;
    skel[:,-1] = 0;

    
    #Find Branch Points
    branchpoints = skeleton_branchpoints(skel);
    bp = np.array(np.where(branchpoints==1))
    
    #Iterate through Branch Points, Remove Unncessary Pixels 
    structure = np.ones((3, 3), dtype=int) 
    for j in range(0, np.shape(bp)[1],1):

        #Box Around Branch Point
        box = skel[int(bp[0,j])-1:int(bp[0,j])+2, int(bp[1,j])-1:int(bp[1,j])+2].copy()
        
        #Before Removing Center
        labeled, ncomponents = label(box, structure)
        
        #After Removing Center
        box[1,1] = 0;     
        labeled, ncomponents_new = label(box, structure)   
        
        #If Unchanged, Remove Center
        if(ncomponents ==ncomponents_new ):
            skel[int(bp[0,j]), int(bp[1,j])] = 0;


    #Find Endpoints        
    endpoints = skeleton_endpoints(skel);
    ep = np.array(np.where(endpoints==1));        
            
    #Separate Branches
    branchpoints = skeleton_branchpoints(skel);
    skel_sep = skel.copy();
    skel_sep[np.where(branchpoints==1)] = 0;     
    
    
    #Create Unique labels for Branches
    labeled, ncomponents = label(skel_sep, structure)
    labeled = labeled*skel_sep;
    unique = np.unique(labeled)
    
    
    #Iterate through Branches
    for unique_label in unique:
        if(unique_label!=0):
            
            #Current Branch
            current = np.zeros(np.shape(labeled));
            current[np.where(labeled==unique_label)] = 1;
            
            
            #If Current Branch Contains Endpoint
            summation= np.sum(np.multiply(current,endpoints))       
            if(np.shape(np.where(labeled==unique_label))[1]<10 and summation>0):
                
                #Remove Small Branches
                skel[np.where(labeled==unique_label)] = 0;


    #Remove  Spurs 
    branchpoints = skeleton_branchpoints(skel);  
    skel[np.where(branchpoints ==1)] = 0;    
    struct1 = ndimage.generate_binary_structure(2, 2)
    skel = ndimage.binary_dilation(skel , structure=struct1)
    skel = skeletonize(skel)


    #Count Branchpoints
    branchpoints = skeleton_branchpoints(skel); 
    labeled, ncomponents = label(branchpoints, structure)
    labeled = labeled*branchpoints;
    count = np.shape(np.unique(labeled))[0];

    #Normalize by Area of SNP
    count = count * np.size(mask_AAR)/np.shape(np.where(mask_AAR==1))[1];
       

    return count;





Nerve Density

In [ ]:
def calculate_nerve_density(mask, mask_AAR):

    #Find Nerve
    nerve = np.shape(np.where(mask==1))[1]

    
    #Nerve Density
    total = np.size(mask)
    density = round(400*400*(nerve/total))

    #Normalize by Area of SNP
    density = density * np.size(mask_AAR)/np.shape(np.where(mask_AAR==1))[1];

    return density;



Nerve Thickness

In [ ]:
def calculate_nerve_thickness(mask):
 
    #Skeletonized Nerve
    skel = skeletonize(mask==1);
    skel = np.shape(np.where(skel==1))[1];

    #Find Nerve 
    nerve = np.shape(np.where(mask==1))[1]


    #Nerve Thickness
    length = np.sqrt(np.size(mask));
    thickness = (nerve/skel) *(400/length);

    return thickness;
    

Nerve Tortuosity

In [ ]:
def calculate_nerve_tortuosity(mask):
    
    
    skel = skeletonize(mask==1);
    branchpoints = skeleton_branchpoints(skel);
    skel[np.where(branchpoints==1)] = 0;

    #Initialize RMS Sum
    tau_C_Sum = 0;
    tau_L_Sum = 0;
    tau_CL_Sum = 0;

    #Iterate through objects of length greater than 1
    islands = np.zeros(np.shape(skel))              
    structure = np.ones((3, 3), dtype=int) 
    labeled, ncomponents = label(skel, structure)
    labeled = labeled*skel;
    unique = np.unique(labeled)
    for unique_label in unique:
        if(unique_label!=0):
            if(np.shape(np.where(labeled==unique_label))[1]>3):

                #Find Endpoints
                current_branch = np.zeros(np.shape(skel));
                current_branch[np.where(labeled==unique_label)] = 1;
                endpoints = skeleton_endpoints(current_branch);
                ep = np.where(endpoints==1);
                y_pos = ep[0];
                x_pos = ep[1]

                #Skip Issues
                if(np.shape(np.where(endpoints==1))[1]==0):
                    continue;
                

                #Find Straight-Line Distance
                Lx = np.sqrt(np.power(y_pos[0]-y_pos[1],2)+np.power(x_pos[0]-x_pos[1],2));

                #Pixels in Branch
                locs = np.where(current_branch==1);
                y_locs = locs[0];
                x_locs = locs[1];

                #Order Pixels in branch
                distances = np.sqrt(np.power(y_locs-y_pos[0],2) + np.power(x_locs-x_pos[0],2));
                idx = np.argsort(distances);
                y_locs = y_locs[idx];
                x_locs = x_locs[idx];



                Lc=0;
                tau_C=0;
                for a in range(1,np.shape(y_locs)[0]):
                    #Calculate First Differential
                    first_x = x_locs[a]-x_locs[a-1];
                    first_y = y_locs[a]-y_locs[a-1];


                    if a>1:
                      
                        #Calculate Second Differential
                        prev_x = x_locs[a-1]-x_locs[a-2];
                        prev_y = y_locs[a-1]-y_locs[a-2];
                        second_x = first_x-prev_x;
                        second_y = first_y-prev_y;
                        Ki = ((first_x*second_y)-(second_x*first_y))/np.power(np.power(first_x,2)+np.power(first_y,2),(3/2));
                    else:
                        Ki=0

                    #Calculate Various Measures
                    Lc = Lc + np.sqrt(np.power(first_x,2)+np.power(first_y,2));
                    tau_C = tau_C + abs(Ki);

                #Weight Different measures
                tau_L = Lc/Lx;
                tau_CL = tau_C/Lc;
                tau_C_Sum = tau_C_Sum + tau_C*np.shape(np.where(labeled==unique_label))[1];
                tau_L_Sum = tau_L_Sum + tau_L*np.shape(np.where(labeled==unique_label))[1];
                tau_CL_Sum = tau_CL_Sum + tau_CL*np.shape(np.where(labeled==unique_label))[1];



    #Normalize by Skel
    skel_count = np.shape(np.where(skel==1))
    skel_count = skel_count[1];  


    #Add to Tortuosity List
    return tau_C_Sum/skel_count;




Fix Mixture

In [ ]:
def fix_mixture_1(prediction, class_1, class_2):

    
    # Class Locations
    binary = np.zeros(np.shape(prediction)) 
    binary[np.where(prediction==class_1)] = 1;
    binary[np.where(prediction==class_2)] = 1;
    
    #Create Labeled Islands
    islands = np.zeros(np.shape(prediction))              
    structure = np.ones((3, 3), dtype=int) 
    labeled, ncomponents = label(binary, structure)
    labeled = labeled*binary;
    
    #Iterate through Islands
    unique = np.unique(labeled)
    for unique_label in unique:
        if(unique_label!=0):
            
            #If more than one class in island
            if(np.unique(prediction[np.where(labeled==unique_label)]).size >1):
                
                                              
                #More abundant class dominates
                count_1 = np.shape(np.where(prediction[np.where(labeled==unique_label)]==class_1))[1];
                count_2 = np.shape(np.where(prediction[np.where(labeled==unique_label)]==class_2))[1];
                if( count_1>count_2):
                    prediction[np.where(labeled==unique_label)] = class_1
                else:
                    prediction[np.where(labeled==unique_label)] = class_2
                    
    return prediction;

In [ ]:
def fix_mixture_2(prediction, class_1, class_2):

    
    # Class Locations
    binary = np.zeros(np.shape(prediction)) 
    binary[np.where(prediction==class_1)] = 1;
    binary[np.where(prediction==class_2)] = 1;
    
    #Create Labeled Islands
    islands = np.zeros(np.shape(prediction))              
    structure = np.ones((3, 3), dtype=int) 
    labeled, ncomponents = label(binary, structure)
    labeled = labeled*binary;
    
    #Iterate through Islands
    unique = np.unique(labeled)
    for unique_label in unique:
        if(unique_label!=0):
            
            #If more than one class in island
            if(np.unique(prediction[np.where(labeled==unique_label)]).size >1):
                
                                              
                #More abundant class dominates
                count_1 = np.shape(np.where(prediction[np.where(labeled==unique_label)]==class_1))[1];
                count_2 = np.shape(np.where(prediction[np.where(labeled==unique_label)]==class_2))[1];
                if( count_1<count_2):
                    prediction[np.where(labeled==unique_label)] = class_2
                    
    return prediction;

Fix Class within Class

In [ ]:
def fix_inner(prediction):

    #Iterate through Classes
    for class_outer in range(1,4):

        # Class Locations
        binary = np.zeros(np.shape(prediction)) 
        binary[np.where(prediction==class_outer)] = 1;
        
        
        #Fill Holes
        binary = ndimage.binary_fill_holes(binary).astype(int)
        binary[np.where(prediction==class_outer)] = 0;
        
        #Create Labeled Islands
        structure = np.ones((3, 3), dtype=int) 
        labeled, ncomponents = label(binary, structure)
        labeled = labeled*binary;
        
        
        #Iterate through Islands
        unique = np.unique(labeled)
        for unique_label in unique:
            if(unique_label!=0):    
                
                #If it doesn't contain background
                if(0 not in np.unique(prediction[np.where(labeled==unique_label)])):
                    prediction[np.where(labeled==unique_label)] = class_outer;
    
    return prediction;

Remove Small Objects

In [ ]:
def remove_small(prediction):

    #Iterate through Classes
    for current_class in range(1,4):

        # Class Locations
        binary = np.zeros(np.shape(prediction)) 
        binary[np.where(prediction==current_class)] = 1;
        
        
        #Create Labeled Islands
        islands = np.zeros(np.shape(prediction))              
        structure = np.ones((3, 3), dtype=int) 
        labeled, ncomponents = label(binary, structure)
        labeled = labeled*binary;

        #Iterate through Islands
        unique = np.unique(labeled)
        for unique_label in unique:
            if(unique_label!=0): 
                
                if(np.shape(np.where(labeled ==unique_label))[1]<8):
                    prediction[np.where(labeled==unique_label)] = 0;
    
    return prediction;

Test Network

In [ ]:


def test_net(python_path, torch_path, img_path, Experiment_Name, Model_Name, tissues, channels, Plot_Figures, Save_Figures):
    
 

    #Cast to Cuda
    CUDA_VISIBLE_DEVICES=2
    cuda = True if torch.cuda.is_available() else False
    if cuda:
        FloatTensor = torch.cuda.FloatTensor
        LongTensor = torch.cuda.LongTensor
    else:
        FloatTensor = torch.FloatTensor
        LongTensor = torch.LongTensor

    #Attempt to use GPU instead of CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');


    #Load Numpy Files
    PID =  np.load(python_path + 'SNP_Net_Groups.npy');  
    SNP = np.load(python_path + 'SNP_Net_Images.npy');
    masks = np.load(python_path + 'SNP_Net_Masks.npy');
    masks_AAR = np.load(python_path  + 'AAR_Net_Masks.npy');  


    #Correct Applanation Artifact Removal
    masks_AAR = (masks_AAR==0);
    masks_AAR = masks_AAR *1.0;
    
    #Weight
    weight_kernel = fspecial_gauss(96, 5);
    
        
    #Initialize
    fscores = np.zeros((np.shape(SNP)[0], tissues));
    sensitivity = np.zeros((np.shape(SNP)[0], tissues));
    specificity = np.zeros((np.shape(SNP)[0], tissues));
    supports = np.zeros((np.shape(SNP)[0], tissues));
    immune_counts = np.zeros((np.shape(SNP)[0], 2));
    neuroma_counts = np.zeros((np.shape(SNP)[0], 2));
    junction_counts = np.zeros((np.shape(SNP)[0], 2));
    nerve_densities = np.zeros((np.shape(SNP)[0], 2));
    nerve_thicknesses = np.zeros((np.shape(SNP)[0], 2));  
    nerve_tortuosities = np.zeros((np.shape(SNP)[0], 2));   
        
    for t in range(0,21):    
   

        #Load Networks
        validation_0 = np.load(torch_path  +'Validation_Dice_'+  Model_Name +'_' + Experiment_Name + '_Group_' + str(t)  +'_0.npy').squeeze();    
        epoch_0 = np.argmax(validation_0[0,60:]) + 60;
        net_0 = SNP_Net(n_channels=channels, n_classes=tissues);
        net_0.load_state_dict(torch.load(torch_path + Model_Name + '_' + Experiment_Name + '_Group_' + str(t) +'_0_Epoch_' +str(epoch_0+1) +'.pth', map_location=device));
        net_0.to(device=device);  
        net_0.eval();                        
        validation_1 = np.load(torch_path  +'Validation_Dice_'+  Model_Name +'_' + Experiment_Name + '_Group_' + str(t)  +'_1.npy').squeeze();    
        epoch_1 =np.argmax(validation_1[0,60:]) + 60;                    
        net_1 = SNP_Net(n_channels=channels, n_classes=tissues);
        net_1.load_state_dict(torch.load(torch_path + Model_Name + '_' + Experiment_Name + '_Group_' + str(t) +'_1_Epoch_' +str(epoch_1+1) +'.pth', map_location=device));
        net_1.to(device=device);  
        net_1.eval();                                 
        validation_2 = np.load(torch_path  +'Validation_Dice_'+  Model_Name +'_' + Experiment_Name + '_Group_' + str(t)  +'_2.npy').squeeze();    
        epoch_2 =np.argmax(validation_2[0,60:]) + 60;                                   
        net_2 = SNP_Net(n_channels=channels, n_classes=tissues);
        net_2.load_state_dict(torch.load(torch_path + Model_Name + '_' + Experiment_Name + '_Group_' + str(t) +'_2_Epoch_' +str(epoch_2+1) +'.pth', map_location=device));
        net_2.to(device=device);  
        net_2.eval();                               
        validation_3 = np.load(torch_path  +'Validation_Dice_'+  Model_Name +'_' + Experiment_Name + '_Group_' + str(t)  +'_3.npy').squeeze();    
        epoch_3 =np.argmax(validation_3[0,60:]) + 60;                                 
        net_3 = SNP_Net(n_channels=channels, n_classes=tissues);
        net_3.load_state_dict(torch.load(torch_path + Model_Name + '_' + Experiment_Name + '_Group_' + str(t) +'_3_Epoch_' +str(epoch_3+1) +'.pth', map_location=device));
        net_3.to(device=device);  
        net_3.eval();                                 
        validation_4 = np.load(torch_path  +'Validation_Dice_'+  Model_Name +'_' + Experiment_Name + '_Group_' + str(t)  +'_4.npy').squeeze();    
        epoch_4 =np.argmax(validation_4[0,60:]) + 60;  
        net_4 = SNP_Net(n_channels=channels, n_classes=tissues);
        net_4.load_state_dict(torch.load(torch_path + Model_Name + '_' + Experiment_Name + '_Group_' + str(t) +'_4_Epoch_' +str(epoch_4+1) +'.pth', map_location=device));
        net_4.to(device=device);  
        net_4.eval();
        
        #Iterate through Images
        for idx in range(0, np.shape(SNP)[0]):

            if(PID[idx] !=t):  
                continue;                
            
            print(idx)
            
            #Select Image
            img = SNP[idx, :,:].copy();
            mask = masks[idx, :,:].copy();
            mask_AAR = masks_AAR[idx, :,:].copy();

            #Normalize
            img = img/np.max(img)
            img = img - 0.5;    

            #Initialize
            collective = np.zeros((tissues,384,384));
            weight = np.zeros((tissues,384,384));

 
            #Iterate through Patches
            for row in range(0,324,36):
                for col in range(0,324,36):

                    for flip_idx in range(0,4):

                        #Select Window
                        cropped_img = img[row:row+96,col:col+96].copy();



                         #Flip Image
                        if(flip_idx >1):
                            cropped_img = np.fliplr(cropped_img).copy();
                        if(flip_idx ==1 or flip_idx ==3):
                            cropped_img = np.flipud(cropped_img).copy();               
                        cropped_img = np.expand_dims(cropped_img,axis = 0);

                        #Cast to Torch
                        input_img = torch.from_numpy(cropped_img).unsqueeze(0).to(device=device, dtype=torch.float32);


                        with torch.no_grad():

                            #Predict the Mask                                         
                            output = net_0(input_img) + net_1(input_img) + net_2(input_img) + net_3(input_img) + net_4(input_img);


                            output = output.cpu().detach().numpy().squeeze();

                            #Multiply with Gaussian Weight
                            mult_output = np.multiply(softmax(output,axis = 0),weight_kernel);   

                            #Flip
                            flip_output = mult_output.copy()
                            if(flip_idx >1):
                                mult_output[0] = np.fliplr(mult_output[0]).copy();
                                mult_output[1] = np.fliplr(mult_output[1]).copy();
                                mult_output[2] = np.fliplr(mult_output[2]).copy();
                                mult_output[3] = np.fliplr(mult_output[3]).copy();
                            if(flip_idx ==1 or flip_idx ==3):
                                mult_output[0] = np.flipud(mult_output[0]).copy();
                                mult_output[1] = np.flipud(mult_output[1]).copy();
                                mult_output[2] = np.flipud(mult_output[2]).copy();
                                mult_output[3] = np.flipud(mult_output[3]).copy();

                            #Add to Collective
                            collective[:,row:row+96,col:col+96] += mult_output
                            weight[:,row:row+96,col:col+96] +=weight_kernel ;


            #Divide by Count
            collective = np.divide(collective, weight);
            prediction = np.argmax(collective, axis = 0);


            #Applanatin Artifact Removal
            prediction = prediction*mask_AAR;


            #Fix Mixture
            prediction = fix_mixture_1(prediction, 2, 3);
            prediction = fix_mixture_2(prediction, 1, 2);

            #Fix Classes within Class
            prediction = fix_inner(prediction);


            #Remove Small Objects
            prediction = remove_small(prediction);



            #Calculate Metrics
            cm = ConfusionMatrix(tissues)
            cm.add(prediction, mask)
            acc_out = segmentation_metrics(cm);
            sens_per_class = cm.get_sensitivity_per_class();
            spec_per_class = cm.get_specificity_per_class();
            sensitivity[idx,0:4] = sens_per_class;
            specificity[idx,0:4] = spec_per_class;       
            temp_1 =   mask.flatten();
            temp_2 =   prediction.flatten();
            x = np.array([0,1,2,3]).astype('float64');
            temp_1 =  np.concatenate((temp_1, x))
            temp_2 =  np.concatenate((temp_2, x))
            precision,recall,fscore,support=score(temp_1,temp_2)
            fscores[idx,0:4] = fscore;
            supports[idx,0:4] = support -1;

            
            #Clinical Metrics: Automatic
            immune_counts[idx,0] = count_immune_cells(prediction, mask_AAR);
            neuroma_counts[idx,0] =  count_neuromas(prediction, mask_AAR); 
            junction_counts[idx,0] =  count_junctions(prediction, mask_AAR); 
            nerve_densities[idx,0] =  calculate_nerve_density(prediction, mask_AAR);
            nerve_thicknesses[idx,0] =  calculate_nerve_thickness(prediction);
            nerve_tortuosities[idx,0] =  calculate_nerve_tortuosity(prediction); 
 
            #Clinical Metrics: Manual
            immune_counts[idx,1] = count_immune_cells(mask, mask_AAR);
            neuroma_counts[idx,1] = count_neuromas(mask, mask_AAR); 
            junction_counts[idx,1] = count_junctions(mask, mask_AAR); 
            nerve_densities[idx,1] =  calculate_nerve_density(mask, mask_AAR);
            nerve_thicknesses[idx,1] = calculate_nerve_thickness(mask); 
            nerve_tortuosities[idx,1] = calculate_nerve_tortuosity(mask);  

            
            #Plot
            if(Plot_Figures):

                #Plot Original Image
                plt.figure;
                plt.title('Image')
                plt.imshow(img,cmap='gray')
                plt.show()        

            if(Plot_Figures or Save_Figures):
                
                #Convert Mask to Color
                color_new_mask_1 = np.zeros((384,384));
                color_new_mask_2 = np.zeros((384,384));
                color_new_mask_3 = np.zeros((384,384));
                color_new_mask_1[np.where(mask==1)] = 255;
                color_new_mask_2[np.where(mask==3)] = 255;
                color_new_mask_3[np.where(mask==2)] = 255;
                color_new_mask = np.zeros((384,384,3));
                color_new_mask[:,:,0] = color_new_mask_1;
                color_new_mask[:,:,1] = color_new_mask_3
                color_new_mask[:,:,2] = color_new_mask_2;
                color_new_mask = np.array(color_new_mask, dtype = 'uint8')
            
            #Plot
            if(Plot_Figures):
           
                #Plot Mask
                plt.figure;
                plt.title('Ground Truth')
                plt.imshow(color_new_mask)
                plt.show()  

            #Save Ground Truth Mask
            if(Save_Figures):
                    fname = 'Manual_' + str(idx) + '.jpg';               
                    save_img = Image.fromarray(color_new_mask);
                    save_img.save(img_path + fname)

            if(Plot_Figures or Save_Figures):
                
                #Convert Mask to Color
                color_new_mask_1 = np.zeros((384,384));
                color_new_mask_2 = np.zeros((384,384));
                color_new_mask_3 = np.zeros((384,384));
                color_new_mask_1[np.where(prediction==1)] = 255;
                color_new_mask_2[np.where(prediction==3)] = 255;
                color_new_mask_3[np.where(prediction==2)] = 255;
                color_new_mask = np.zeros((384,384,3));
                color_new_mask[:,:,0] = color_new_mask_1;
                color_new_mask[:,:,1] = color_new_mask_3
                color_new_mask[:,:,2] = color_new_mask_2;
                color_new_mask = np.array(color_new_mask, dtype = 'uint8')

             #Plot
            if(Plot_Figures):  
                
                #Plot Mask
                plt.figure;
                plt.title('Automatic')
                plt.imshow(color_new_mask)
                plt.show()

            #Save Ground Truth Mask
            if(Save_Figures):
                fname = 'Automatic_' + str(idx) + '.jpg';
                save_img = Image.fromarray(color_new_mask);
                save_img.save(img_path + fname)                   
 

 

    return fscores, sensitivity, specificity, supports, immune_counts, neuroma_counts, junction_counts, nerve_densities, nerve_thicknesses, nerve_tortuosities;

Main

In [ ]:
#Parameters
Experiment_Name = 'Original';
Model_Name = 'SNP_Net';
tissues = 4;
channels = 1;
Plot_Figures = True;
Save_Figures = True;


#Path
python_path = r'/hpc/group/viplab/zzz3/SNP_Segmentation/Files/Python/SNP-Net/';
torch_path = r'/hpc/group/viplab/zzz3/SNP_Segmentation/Files/Torch/SNP-Net/';
img_path = r'/hpc/group/viplab/zzz3/SNP_Segmentation/Files/Images/SNP-Net/';


#Test AAR-Net
fscores, sensitivity, specificity, supports, immune_counts, neuroma_counts, junction_counts, nerve_densities, nerve_thicknesses, nerve_tortuosities = test_net(python_path, torch_path, img_path, Experiment_Name, Model_Name, tissues, channels, Plot_Figures, Save_Figures);

Table 1 Metrics

In [ ]:

#Zero Out
fscores[np.where(supports==0)] = 0;
sensitivity[np.where(supports==0)] = 0;


#DSC
print('=====================================================')
print('DSC:')
avg, stDev = weighted_avg_and_std(fscores[:,0] ,supports[:,0]);
print('Background: ' + str(round(avg, 3)) + '; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores[:,1] ,supports[:,1]);
print('Nerve: ' + str(round(avg, 3)) + '; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores[:,3] ,supports[:,3]);
print('Neuroma: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores[:,2] ,supports[:,2]);
print('Immune: ' + str(round(avg, 3)) + '; ' + str(round(stDev, 3)))
print('=====================================================')

#Sensitivity
print('Sensitivity:')
avg, stDev = weighted_avg_and_std(sensitivity[:,0] ,supports[:,0]);
print('Background: ' + str(round(avg, 3)) + '; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity[:,1] ,supports[:,1]);
print('Nerve: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity[:,3] ,supports[:,3]);
print('Neuroma: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity[:,2] ,supports[:,2]);
print('Immune: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
print('=====================================================')

#Specificity
print('Specificity:')
weight = supports[:,1] + supports[:,2] + supports[:,3];
avg, stDev = weighted_avg_and_std(specificity[:,0] ,weight);
print('Background: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
weight = supports[:,0] + supports[:,2] + supports[:,3];
avg, stDev = weighted_avg_and_std(specificity[:,1] ,weight);
print('Nerve: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
weight = supports[:,0] + supports[:,1] + supports[:,2];
avg, stDev = weighted_avg_and_std(specificity[:,3] ,weight);
print('Neuroma: ' + str(round(avg, 4)) +'; ' + str(round(stDev, 4)))
weight = supports[:,0] + supports[:,1] + supports[:,3];
avg, stDev = weighted_avg_and_std(specificity[:,2] ,weight);
print('Immune: ' + str(round(avg, 4)) +'; ' + str(round(stDev, 4)))
print('=====================================================')



Table 2 Metrics

In [ ]:
#Nerve Density
print('=====================================================')
print('Nerve Density:')
avg = np.nanmean(nerve_densities[:,0]); stDev = np.nanstd(nerve_densities[:,0]);
print('Automatic: ' + str(round(avg))+ '; ' + str(round(stDev)))
avg = np.mean(nerve_densities[:,1]); stDev = np.std(nerve_densities[:,1]);
print('Manual: ' + str(round(avg)) +'; ' + str(round(stDev)))
print('=====================================================')

#Average Nerve Thickness
print('Average Nerve Thickness:')
avg = np.nanmean(nerve_thicknesses[:,0]); stDev = np.nanstd(nerve_thicknesses[:,0]);
print('Automatic: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
avg = np.nanmean(nerve_thicknesses[:,1]); stDev = np.nanstd(nerve_thicknesses[:,1]);
print('Manual: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
print('=====================================================')

#Average Nerve Segment Tortuosity
print('Average Nerve Segment Tortuosity:')
avg = np.nanmean(nerve_tortuosities[:,0]); stDev = np.nanstd(nerve_tortuosities[:,0]);
print('Automatic: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
avg = np.nanmean(nerve_tortuosities[:,1]); stDev = np.nanstd(nerve_tortuosities[:,1]);
print('Manual: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
print('=====================================================')

#Junction Point Density
print('Junction Point Density:')
avg = np.nanmean(junction_counts[:,0]); stDev = np.nanstd(junction_counts[:,0]);
print('Automatic: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
avg = np.nanmean(junction_counts[:,1]); stDev = np.nanstd(junction_counts[:,1]);
print('Manual: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
print('=====================================================')
             
#Neuroma Density
print('Neuroma Density:')
avg = np.nanmean(neuroma_counts[:,0]); stDev = np.nanstd(neuroma_counts[:,0]);
print('Automatic: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
avg = np.nanmean(neuroma_counts[:,1]); stDev = np.nanstd(neuroma_counts[:,1]);
print('Manual: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))             
print('=====================================================') 
             
#Immune Cell Density
print('Immune Cell Density:')
avg = np.nanmean(immune_counts[:,0]); stDev = np.nanstd(immune_counts[:,0]);
print('Automatic: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1)))
avg = np.nanmean(immune_counts[:,1]); stDev = np.nanstd(immune_counts[:,1]);
print('Manual: ' + str(round(avg, 1)) +'; ' + str(round(stDev, 1))) 
print('=====================================================')


Intraclass Correlation

In [ ]:
#Initialize Intraclass Correlation
a = np.arange(0,207);
aa = np.concatenate((a,a), axis = 0);
b0 = np.zeros((207,1)).squeeze();
b1 = np.ones((207,1)).squeeze();
bb = np.concatenate((b0,b1), axis = 0);

cc = np.concatenate((nerve_densities[:,0], nerve_densities[:,1]),axis=0)
#cc = np.concatenate((nerve_thicknesses[:,0], nerve_thicknesses[:,1]),axis=0)
#cc = np.concatenate((nerve_tortuosities[:,0], nerve_tortuosities[:,1]),axis=0)
#cc = np.concatenate((junction_counts[:,0], junction_counts[:,1]),axis=0)
#cc = np.concatenate((neuroma_counts[:,0], neuroma_counts[:,1]),axis=0) 
#cc = np.concatenate((immune_counts[:,0], immune_counts[:,1]),axis=0)  

df = pd.DataFrame({'exam': aa, 'judge': bb, 'rating': cc})
icc = pg.intraclass_corr(data=df, targets='exam', raters='judge', ratings='rating')
icc.set_index('Type')

Table 3 Metrics

In [ ]:

#Load Numpy
other_grader = np.load(python_path + 'SNP_Net_Other_Grader.npy'); 
masks_other = np.load(python_path + 'SNP_Net_Masks_Other.npy');
masks = np.load(python_path + 'SNP_Net_Masks.npy');
masks_original = masks[other_grader];

#Automatic versus Reader 1
fscores_original = fscores[other_grader];
sensitivity_original = sensitivity[other_grader];
specificity_original = specificity[other_grader];
supports_original = supports[other_grader];



print('=====================================================')
print('SNP-Net versus Reader 1')
print('=====================================================')

#DSC
print('DSC:')
avg, stDev = weighted_avg_and_std(fscores_original[:,0] ,supports_original[:,0]);
print('Background: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores_original[:,1] ,supports_original[:,1]);
print('Nerve: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores_original[:,3] ,supports_original[:,3]);
print('Neuroma: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores_original[:,2] ,supports_original[:,2]);
print('Immune: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
print('=====================================================')

#Sensitivity
print('Sensitivity:')
avg, stDev = weighted_avg_and_std(sensitivity_original[:,0] ,supports_original[:,0]);
print('Background: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity_original[:,1] ,supports_original[:,1]);
print('Nerve: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity_original[:,3] ,supports_original[:,3]);
print('Neuroma: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity_original[:,2] ,supports_original[:,2]);
print('Immune: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
print('=====================================================')

#Specificity
print('Specificity:')
weight = supports_original[:,1] + supports_original[:,2] + supports_original[:,3];
avg, stDev = weighted_avg_and_std(specificity_original[:,0] ,weight);
print('Background: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
weight = supports_original[:,0] + supports_original[:,2] + supports_original[:,3];
avg, stDev = weighted_avg_and_std(specificity_original[:,1] ,weight);
print('Nerve: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
weight = supports_original[:,0] + supports_original[:,1] + supports_original[:,2];
avg, stDev = weighted_avg_and_std(specificity_original[:,3] ,weight);
print('Neuroma: ' + str(round(avg, 4)) +'; ' + str(round(stDev, 4)))
weight = supports_original[:,0] + supports_original[:,1] + supports_original[:,3];
avg, stDev = weighted_avg_and_std(specificity_original[:,2] ,weight);
print('Immune: ' + str(round(avg, 4))+ '; ' + str(round(stDev, 4)))






#Initialize
fscores_other = np.zeros((np.shape(masks_other)[0], tissues));
sensitivity_other = np.zeros((np.shape(masks_other)[0], tissues));
specificity_other = np.zeros((np.shape(masks_other)[0], tissues));
supports_other = np.zeros((np.shape(masks_other)[0], tissues));

#Iterate
for idx in range(0, np.shape(masks_other)[0]):
    
    #Mask
    mask_original = masks_original[idx,:,:].squeeze();
    mask_other = masks_other[idx,:,:].squeeze();
    
    
    #Calculate Metrics
    cm = ConfusionMatrix(tissues)
    cm.add(mask_other, mask_original)
    acc_out = segmentation_metrics(cm);
    sens_per_class = cm.get_sensitivity_per_class();
    spec_per_class = cm.get_specificity_per_class();
    sensitivity_other[idx,0:4] = sens_per_class;
    specificity_other[idx,0:4] = spec_per_class;       
    temp_1 =   mask_original.flatten();
    temp_2 =   mask_other.flatten();
    x = np.array([0,1,2,3]).astype('float64');
    temp_1 =  np.concatenate((temp_1, x))
    temp_2 =  np.concatenate((temp_2, x))
    precision,recall,fscore,support=score(temp_1,temp_2)
    fscores_other[idx,0:4] = fscore;
    supports_other[idx,0:4] = support -1; 
    

#Zero Out
fscores_other[np.where(supports_other==0)] = 0;
sensitivity_other[np.where(supports_other==0)] = 0;

    
print('=====================================================')
print('Reader 2 & 3 versus Reader 1')
print('=====================================================')
    
#DSC
print('DSC:')
avg, stDev = weighted_avg_and_std(fscores_other[:,0] ,supports_other[:,0]);
print('Background: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores_other[:,1] ,supports_other[:,1]);
print('Nerve: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores_other[:,3] ,supports_other[:,3]);
print('Neuroma: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(fscores_other[:,2] ,supports_other[:,2]);
print('Immune: ' + str(round(avg, 3))+ '; ' + str(round(stDev, 3)))
print('=====================================================')

#Sensitivity
print('Sensitivity:')
avg, stDev = weighted_avg_and_std(sensitivity_other[:,0] ,supports_other[:,0]);
print('Background: ' + str(round(avg, 3))+ '; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity_other[:,1] ,supports_other[:,1]);
print('Nerve: ' + str(round(avg, 3))+ '; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity_other[:,3] ,supports_other[:,3]);
print('Neuroma: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
avg, stDev = weighted_avg_and_std(sensitivity_other[:,2] ,supports_other[:,2]);
print('Immune: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
print('=====================================================')


#Specificity
print('Specificity:')
weight = supports_other[:,1] + supports_other[:,2] + supports_other[:,3];
avg, stDev = weighted_avg_and_std(specificity_other[:,0] ,weight);
print('Background: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
weight = supports_other[:,0] + supports_other[:,2] + supports_other[:,3];
avg, stDev = weighted_avg_and_std(specificity_other[:,1] ,weight);
print('Nerve: ' + str(round(avg, 3)) +'; ' + str(round(stDev, 3)))
weight = supports_other[:,0] + supports_other[:,1] + supports_other[:,2];
avg, stDev = weighted_avg_and_std(specificity_other[:,3] ,weight);
print('Neuroma: ' + str(round(avg, 4)) +'; ' + str(round(stDev, 4)))
weight = supports_other[:,0] + supports_other[:,1] + supports_other[:,3];
avg, stDev = weighted_avg_and_std(specificity_other[:,2] ,weight);
print('Immune: ' + str(round(avg, 4)) + '; ' + str(round(stDev, 4)))




    
#Calculate Difference
fscores_diff = fscores_original - fscores_other;
sensitivity_diff = sensitivity_original - sensitivity_other;
specificity_diff = specificity_original - specificity_other;


#Ignore Certain Values
fscores_diff[np.where(supports_other==0.0)] = 'NaN';
sensitivity_diff[np.where(supports_other==0.0)] = 'NaN';



print('=====================================================')
print('P-Values: SNP-Net Greater')
print('=====================================================')


#DSC
print('DSC:')
res = wilcoxon(fscores_diff[:,0], alternative='greater', nan_policy='omit')
print('Background: ' + str(round(res.pvalue, 3)))
res = wilcoxon(fscores_diff[:,1], alternative='greater', nan_policy='omit')
print('Nerve: ' + str(round(res.pvalue, 3)))
res = wilcoxon(fscores_diff[:,3], alternative='greater', nan_policy='omit')
print('Neuroma: ' + str(round(res.pvalue, 3)) )
res = wilcoxon(fscores_diff[:,2], alternative='greater', nan_policy='omit')
print('Immune: ' + str(round(res.pvalue, 3)) )
print('=====================================================')

#Sensitivity
print('Sensitivity:')
res = wilcoxon(sensitivity_diff[:,0], alternative='greater', nan_policy='omit')
print('Background: ' + str(round(res.pvalue, 3)))
res = wilcoxon(sensitivity_diff[:,1], alternative='greater', nan_policy='omit')
print('Nerve: ' + str(round(res.pvalue, 3)))
res = wilcoxon(sensitivity_diff[:,3], alternative='greater', nan_policy='omit')
print('Neuroma: ' + str(round(res.pvalue, 3)) )
res = wilcoxon(sensitivity_diff[:,2], alternative='greater', nan_policy='omit')
print('Immune: ' + str(round(res.pvalue, 3)) )
print('=====================================================')

#Specificity
print('Specificity:')
res = wilcoxon(specificity_diff[:,0], alternative='greater', nan_policy='omit')
print('Background: ' + str(round(res.pvalue, 3)))
res = wilcoxon(specificity_diff[:,1], alternative='greater', nan_policy='omit')
print('Nerve: ' + str(round(res.pvalue, 3)))
res = wilcoxon(specificity_diff[:,3], alternative='greater', nan_policy='omit')

print('Neuroma: ' + str(round(res.pvalue, 3)) )
res = wilcoxon(specificity_diff[:,2], alternative='greater', nan_policy='omit')
print('Immune: ' + str(round(res.pvalue, 3)) )
print('=====================================================')


fscores_diff = fscores_other - fscores_original;
sensitivity_diff = sensitivity_other - sensitivity_original;
specificity_diff = specificity_other - specificity_original;



#Ignore Certain Values
fscores_diff[np.where(supports_other==0.0)] = 'NaN';
sensitivity_diff[np.where(supports_other==0.0)] = 'NaN';


print('=====================================================')
print('P-Values: Other Graders Greater')
print('=====================================================')


#DSC
print('DSC:')
res = wilcoxon(fscores_diff[:,0], alternative='greater', nan_policy='omit')
print('Background: ' + str(round(res.pvalue, 3)))
res = wilcoxon(fscores_diff[:,1], alternative='greater', nan_policy='omit')
print('Nerve: ' + str(round(res.pvalue, 3)))
res = wilcoxon(fscores_diff[:,3], alternative='greater', nan_policy='omit')
print('Neuroma: ' + str(round(res.pvalue, 3)) )
res = wilcoxon(fscores_diff[:,2], alternative='greater', nan_policy='omit')
print('Immune: ' + str(round(res.pvalue, 3)) )
print('=====================================================')

#Sensitivity
print('Sensitivity:')
res = wilcoxon(sensitivity_diff[:,0], alternative='greater', nan_policy='omit')
print('Background: ' + str(round(res.pvalue, 3)))
res = wilcoxon(sensitivity_diff[:,1], alternative='greater', nan_policy='omit')
print('Nerve: ' + str(round(res.pvalue, 3)))
res = wilcoxon(sensitivity_diff[:,3], alternative='greater', nan_policy='omit')
print('Neuroma: ' + str(round(res.pvalue, 3)) )
res = wilcoxon(sensitivity_diff[:,2], alternative='greater', nan_policy='omit')
print('Immune: ' + str(round(res.pvalue, 3)) )
print('=====================================================')

#Specificity
print('Specificity:')
res = wilcoxon(specificity_diff[:,0], alternative='greater', nan_policy='omit')
print('Background: ' + str(round(res.pvalue, 3)))
res = wilcoxon(specificity_diff[:,1], alternative='greater', nan_policy='omit')
print('Nerve: ' + str(round(res.pvalue, 3)))
res = wilcoxon(specificity_diff[:,3], alternative='greater', nan_policy='omit')

print('Neuroma: ' + str(round(res.pvalue, 3)) )
res = wilcoxon(specificity_diff[:,2], alternative='greater', nan_policy='omit')
print('Immune: ' + str(round(res.pvalue, 3)) )
print('=====================================================')